## If you don't have nltk installed, run the following command:
 

In [2]:
# !pip install nltk


# import libraries

In [4]:
import pandas as pd
import numpy as np
import scipy
import random
import math
import nltk
import sklearn

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

## Download stopwords

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## import dataset

In [8]:
 
article_df=pd.read_csv(fr"D:\data science\Machine Learning\New folder\recommand system\end-to-end-recommender-system-main\end-to-end-recommender-system-main\archive\shared_articles.csv")
interaction_df=pd.read_csv(fr"D:\data science\Machine Learning\New folder\recommand system\end-to-end-recommender-system-main\end-to-end-recommender-system-main\archive\users_interactions.csv")


In [9]:
article_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [10]:
article_df['eventType'].unique()

array(['CONTENT REMOVED', 'CONTENT SHARED'], dtype=object)

## Filter articles

In [12]:
article_df=article_df[article_df['eventType']=='CONTENT SHARED']

In [13]:
article_df.shape

(3047, 13)

In [14]:
interaction_df['eventType'].unique()

array(['VIEW', 'FOLLOW', 'BOOKMARK', 'LIKE', 'COMMENT CREATED'],
      dtype=object)

## Feature engineering

### filling the users-tims matrix

In [17]:
eventType_strength = {'VIEW':1.0, 'FOLLOW':3.0, 'BOOKMARK':2.5, 'LIKE':2.0, 'COMMENT CREATED':4.0}
interaction_df['eventType_strength'] = interaction_df['eventType'].map(eventType_strength)

In [18]:
interaction_df['eventType_strength']

0        1.0
1        1.0
2        1.0
3        3.0
4        1.0
        ... 
72307    2.0
72308    1.0
72309    1.0
72310    1.0
72311    1.0
Name: eventType_strength, Length: 72312, dtype: float64

## Keep only users with at least 5 interactions

In [20]:
users_interactions_count_df = interaction_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
interactions_df = interaction_df.merge(users_with_enough_interactions_df, how='right', on='personId')
print(f"users with as least 5 interac {len(users_with_enough_interactions_df)}")

users with as least 5 interac 1140


In [21]:
print(f"of interaction from users with at leaast 5 inter {len(interactions_df)}")

of interaction from users with at leaast 5 inter 69868


In [22]:
interactions_df.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventType_strength
0,1463138398,VIEW,7516228655554309785,-9223121837663643404,-4482197405545551645,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
1,1463656314,VIEW,3041906492387035176,-9223121837663643404,-7824685088995468735,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
2,1464113091,VIEW,-3750879736572068916,-9223121837663643404,-2774275024909061125,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
3,1462283851,VIEW,-730957269757756529,-9223121837663643404,2625340673871268120,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
4,1462452127,VIEW,-8949113594875411859,-9223121837663643404,-3673331845456357462,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0


In [23]:
interactions_df.groupby(['eventType','eventType_strength']).size()

eventType        eventType_strength
BOOKMARK         2.5                    2420
COMMENT CREATED  4.0                    1579
FOLLOW           3.0                    1379
LIKE             2.0                    5625
VIEW             1.0                   58865
dtype: int64

## Apply log transformation to smooth the distribution 

In [25]:
import math 
def smooth_user_perferece(x):
    return math.log(1+x,2)
interactionFullDf=interactions_df.groupby(['personId','contentId'])['eventType_strength'].sum().apply(smooth_user_perferece).reset_index()
print(f"# of unique user'item interaction{len(interactionFullDf)}")
interactionFullDf.head(10)

# of unique user'item interaction39106


,personId,contentId,eventType_strength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925
5,-9223121837663643404,-7331393944609614247,1.000000
6,-9223121837663643404,-6872546942144599345,1.000000
7,-9223121837663643404,-6728844082024523434,1.000000
8,-9223121837663643404,-6590819806697898649,1.000000
9,-9223121837663643404,-6558712014192834002,1.584963


## Clean text data

In [27]:
def clean_text(text):
    try:
        return text.lower().replace("\n", " ").replace("\"", "").strip()
    except:
        return ""

article_df['title'] = article_df['title'].apply(clean_text)
article_df['text'] = article_df['text'].apply(clean_text)

## item profiles

In [29]:
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.005, max_df=0.6,
                              max_features=6000, stop_words=stopwords_list)
tfidf_matrix = vectorizer.fit_transform(article_df['title'] + " " + article_df['text'])
item_ids = article_df['contentId'].tolist()


In [30]:
tfidf_matrix.shape

(3047, 6000)

## Build train and test sets

In [32]:
interactions_train_df, interactions_test_df = train_test_split(interactionFullDf, stratify=interactionFullDf['personId'], test_size=0.3, random_state=42)

In [33]:
interactions_train_df[interactions_train_df['personId']==3609194402293569455].groupby('personId').size()

personId
3609194402293569455    673
dtype: int64

In [34]:
interactions_test_df[interactions_test_df['personId']==3609194402293569455].groupby('personId').size()

personId
3609194402293569455    288
dtype: int64

## Build user profiles

In [36]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    return tfidf_matrix[idx:idx + 1]

In [37]:
def get_item_profiles(ids):
    return scipy.sparse.vstack([get_item_profile(x) for x in ids])


In [38]:
def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    user_item_strengths = np.array(interactions_person_df['eventType_strength']).reshape(-1, 1)
    weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    return normalize(np.asarray(weighted_avg))
 
 

In [39]:
def build_users_profiles():
    interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'].isin(article_df['contentId'])].set_index('personId')
    return {person_id: build_users_profile(person_id, interactions_indexed_df) for person_id in interactions_indexed_df.index.unique()}


In [40]:
user_profiles=build_users_profiles()
len(user_profiles)




1140

In [41]:
list(user_profiles.keys())[:5] 

[3937943558206985686,
 1874422396201148365,
 5598537709124463353,
 -4165818767652094649,
 2416280733544962613]

In [42]:
# show an example for user profiles >> random user 
tfidf_feature_names = vectorizer.get_feature_names_out()
myprofile = user_profiles[ 2416280733544962613]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names,user_profiles[ 2416280733544962613].flatten().tolist()),key = lambda x : -x[1])[:20],
            columns = ['token','relevance'])

(1, 6000)


,token,relevance
0,google,0.195060
1,digital,0.152890
2,data,0.146311
3,drupal,0.128583
4,new,0.127767
5,cloud,0.121760
6,company,0.121182
7,business,0.113361
8,companies,0.111871
9,learning,0.110029


## Content-based Filtering Model

In [44]:
class ContentBasedRecommender:
    ModelName = 'Content_Based'

    def __init__(self, item_df=None):
        self.item_ids = item_ids
        self.item_df = item_df

    def get_model_name(self):
        return self.ModelName

    def _get_similar_items_to_user_profile(self, person_id, topn=500):
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_items = sorted([(item_ids[i], cosine_similarities[0, i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items

    def _recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        filtered_items = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        recommendations_df = pd.DataFrame(filtered_items, columns=['contentId', 'recStrength']).head(topn)
        if verbose:
            if self.item_df is None:
                raise Exception('item_df is required in verbose mode')
            recommendations_df = recommendations_df.merge(self.item_df, how='left', on='contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]
        return recommendations_df

    

In [45]:
content_model = ContentBasedRecommender(article_df)

## model evalutor

In [47]:
# to speed up the searches during evaluation
interactions_full_indexed_df = interactionFullDf.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')


In [48]:
# top_n accuracy metrics consts
EVAl_RandomSample_NON_Interacted_ITEMS = 100
class ModelEvaluator:
    def get_items_interacted(self, person_id, interaction_df):
        interacted_items = interaction_df.loc[person_id]['contentId']
        return set(interacted_items if isinstance(interacted_items, pd.Series) else [interacted_items])

    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = self.get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(article_df['contentId'])
        non_interacted_items = all_items - interacted_items
        sample_size = min(len(non_interacted_items), sample_size)  # ✅ إصلاح هنا
        random.seed(seed)
        return set(random.sample(list(non_interacted_items), sample_size))

    def _verify_hit_top_n(self, item_id, recommended_items, topn):
        try:
            index = next(i for i, c in enumerate(recommended_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(topn))
        return hit, index

    def evaluate_model_for_user(self, model, person_id):
        test_interactions = interactions_test_indexed_df.loc[person_id]
        if isinstance(test_interactions, pd.Series):
            interacted_items = {test_interactions['contentId']}
        else:
            interacted_items = set(test_interactions['contentId'])

        recommended_df = model._recommend_items(person_id, items_to_ignore=self.get_items_interacted(person_id, interactions_train_indexed_df), topn=500)
        valid_recommendations = []
        hits_at_5 = hits_at_10 = 0

        for item_id in interacted_items:
            non_interacted_sample = self.get_not_interacted_items_sample(person_id, 100, seed=item_id % (2 ** 32))
            test_items = non_interacted_sample.union({item_id})
            recs = recommended_df[recommended_df['contentId'].isin(test_items)]['contentId'].values

            hit5, _ = self._verify_hit_top_n(item_id, recs, 5)
            hit10, _ = self._verify_hit_top_n(item_id, recs, 10)

            hits_at_5 += hit5
            hits_at_10 += hit10

        return {
            'hits@5_count': hits_at_5,
            'hits@10_count': hits_at_10,
            'interacted_count': len(interacted_items)
        }

    def evaluate_model(self, model):
        people_metrics = []
        for idx, person_id in enumerate(interactions_test_indexed_df.index.unique().values):
            metrics = self.evaluate_model_for_user(model, person_id)
            metrics['_person_id'] = person_id
            people_metrics.append(metrics)
        detailed_df = pd.DataFrame(people_metrics)

        recall_at_5 = detailed_df['hits@5_count'].sum() / detailed_df['interacted_count'].sum()
        recall_at_10 = detailed_df['hits@10_count'].sum() / detailed_df['interacted_count'].sum()

        return {
            'modelName': model.get_model_name(),
            'recall@5': recall_at_5,
            'recall@10': recall_at_10
        }, detailed_df

    

## Prepare for evaluation

In [50]:
interactions_full_indexed_df = interactionFullDf.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [51]:
evaluator = ModelEvaluator()
cb_metrics, cb_details = evaluator.evaluate_model(content_model)
print("Global Metrics:\n", cb_metrics)
cb_details.head(10)

Global Metrics:
 {'modelName': 'Content_Based', 'recall@5': 0.1758438458915786, 'recall@10': 0.27088305489260145}


,hits@5_count,hits@10_count,interacted_count,_person_id
0,3,4,35,-4028919343899978105
1,6,20,77,3829784524040647339
2,2,3,22,-7136511247670430002
3,26,47,194,-1032019229384696495
4,0,0,2,-2987910693129502800
5,2,5,26,-1863544689523892600
6,0,1,5,-2376088187709738156
7,11,22,104,692689608292948411
8,22,35,120,-3596626804281480007
9,0,0,1,-4885223078646518099


In [52]:
evaluator.get_items_interacted( 1874422396201148365,interactions_test_indexed_df)

{-8992803137960175254,
 -4700714777342103147,
 1959495508923903948,
 6031953227014493100,
 6519443272707698315}

In [53]:
import pickle
pickle.dump(item_ids, open(r"D:\data science\project kaggle\item_ids.p", "wb"))
pickle.dump(user_profiles, open(r"D:\data science\project kaggle\user_profiles.p", "wb"))
pickle.dump(tfidf_matrix, open(r"D:\data science\project kaggle\tfidf_matrix.p", "wb"))
pickle.dump(content_model, open(r"D:\data science\project kaggle\recommend_model.p", "wb"))  